# Enriquecimiento de la capa Plata

En esta notebook trabajaremos sobre los datos horarios previamente generados y la capa Plata intermedia con el objetivo de **enriquecer y completar la información** antes de su uso para análisis avanzados (clustering, PCA, reglas de asociación, etc.).

### Las principales tareas realizadas son:
- Detección de fechas y horas con datos faltantes.
- Imputación de valores `NaN` basados en el promedio del mismo horario del día anterior y posterior.
- Comparación entre el dataset original y el imputado.
- Exportación del dataset horario imputado.
- Verificación final de la cobertura completa de fechas por estación.

## Importar las librerías necesarias

In [1]:
import pandas as pd
from pathlib import Path
import warnings

# Deshabilitar warnings futuros
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ajustar el ancho máximo para impresión en consola
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 300)         # Ajustar a un ancho suficiente en consola
pd.set_option('display.max_colwidth', None) # Evitar recortes en contenido de celdas

print("Importación de librerías completada.")

Importación de librerías completada.


## Configuración de paths y carpetas del proyecto

In [2]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'
PLATA_DIR = Path("../data/plata")

archivo_plata = PLATA_DIR / "dataset_plata_inicial.csv"
archivo_horario = PLATA_DIR / "horario_archivo.csv"

print("Iniciación de carpetas del proyecto completada.")

Iniciación de carpetas del proyecto completada.


## Carga del dataset y verificación de estructura

In [3]:
# Cargar el dataset diario
try:
    df_plata = pd.read_csv(archivo_plata, parse_dates=["FECHA"])
    print("Dataset diario cargado correctamente")
except FileNotFoundError:
    print("El archivo diario no fue encontrado")

# Cargar el dataset horario
try:
    df_horario = pd.read_csv(archivo_horario, parse_dates=["FECHA_HORA"])
    print("Dataset horario cargado correctamente")
except FileNotFoundError:
    print("El archivo horario no fue encontrado")

# Vista preliminar
print("\n Dataset diario:")
df_plata.info()
print("\n")
print(df_plata.head())

print("\n Dataset horario:")
df_horario.info()
print("\n")
print(df_horario.head())

Dataset diario cargado correctamente
Dataset horario cargado correctamente

 Dataset diario:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2110 entries, 0 to 2109
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ESTACION              2110 non-null   object        
 1   FECHA                 2110 non-null   datetime64[ns]
 2   TEMP_MEAN             2110 non-null   float64       
 3   TEMP_MIN              2110 non-null   float64       
 4   TEMP_MAX              2110 non-null   float64       
 5   PNM_MEAN              2110 non-null   float64       
 6   PNM_MIN               2110 non-null   float64       
 7   PNM_MAX               2110 non-null   float64       
 8   HUM_MEAN              2110 non-null   float64       
 9   HUM_MIN               2110 non-null   int64         
 10  HUM_MAX               2110 non-null   int64         
 11  WIND_DIR_MEAN         2110 non-null   flo

Este paso permite validar la estructura general, tipos de datos y posibles columnas faltantes tanto en el dataset diario como en el horario. Si todo está correcto, avanzaremos con el enriquecimiento.

## Detección y análisis de fechas faltantes

Una vez verificada la estructura del dataset diario, procedemos a identificar si existen fechas faltantes en la serie por estación. 

Esto nos permitirá decidir estrategias para tratar los días sin registros, como imputación o exclusión.


In [4]:
# Generar el rango completo de fechas esperadas
fechas_totales = pd.date_range(start=df_plata['FECHA'].min(), end=df_plata['FECHA'].max(), freq='D')

# Obtener todas las combinaciones posibles de fecha y estación
estaciones = df_plata['ESTACION'].unique()
index_completo = pd.MultiIndex.from_product([fechas_totales, estaciones], names=['FECHA', 'ESTACION'])

# Reindexar para insertar NaNs explícitos en las fechas faltantes
df_plata = df_plata.set_index(['FECHA', 'ESTACION']).reindex(index_completo).reset_index()

# Verificar fechas faltantes (para exportar listado)
faltantes = df_plata[df_plata.isnull().any(axis=1)][['ESTACION', 'FECHA']]

if not faltantes.empty:
    faltantes.to_csv(PLATA_DIR / "fechas_faltantes.txt", index=False, sep='\t')
    print("Fechas faltantes exportadas a:", PLATA_DIR / "fechas_faltantes.txt")
else:
    print("No se encontraron fechas faltantes")

# Mostrar ejemplo si hay faltantes
print(faltantes.head())

Fechas faltantes exportadas a: ../data/plata/fechas_faltantes.txt
                    ESTACION      FECHA
725          CORRIENTES AERO 2024-10-24
726                ITUZAINGO 2024-10-24
727     MERCEDES AERO (CTES) 2024-10-24
728       MONTE CASEROS AERO 2024-10-24
729  PASO DE LOS LIBRES AERO 2024-10-24


Esta estrategia asegura que cada estación tenga una fila para cada fecha del rango, incluso si originalmente no había registros ese día. Esto deja los valores faltantes como `NaN`, que luego se tratarán.

## Tratamiento de valores nulos

Luego de verificar fechas faltantes, analizamos los valores `NaN` dentro del dataset actual para decidir estrategias de imputación o tratamiento.

### Tratamiento de datos faltantes en el dataset diario

In [5]:
# Visualizar cantidad de nulos por columna
print("\nValores nulos por columna:")
print(df_plata.isnull().sum())

# Calcular porcentaje de nulos por columna
porcentaje_nulos = df_plata.isnull().mean() * 100
print("\nPorcentaje de valores nulos:")
print(porcentaje_nulos.round(2))


Valores nulos por columna:
FECHA                    0
ESTACION                 0
TEMP_MEAN               20
TEMP_MIN                20
TEMP_MAX                20
PNM_MEAN                20
PNM_MIN                 20
PNM_MAX                 20
HUM_MEAN                20
HUM_MIN                 20
HUM_MAX                 20
WIND_DIR_MEAN           20
WIND_DIR_MIN            20
WIND_DIR_MAX            20
WIND_SPEED_MEAN         20
WIND_SPEED_MIN          20
WIND_SPEED_MAX          20
TEMP_MEAN_NORM          20
PNM_MEAN_NORM           20
HUM_MEAN_NORM           20
WIND_DIR_MEAN_NORM      20
WIND_SPEED_MEAN_NORM    20
dtype: int64

Porcentaje de valores nulos:
FECHA                   0.00
ESTACION                0.00
TEMP_MEAN               0.94
TEMP_MIN                0.94
TEMP_MAX                0.94
PNM_MEAN                0.94
PNM_MIN                 0.94
PNM_MAX                 0.94
HUM_MEAN                0.94
HUM_MIN                 0.94
HUM_MAX                 0.94
WIND_DIR_MEAN   

Una vez identificadas las columnas afectadas, proponemos distintas estrategias para completar los datos:

### Relleno con forward fill por estación

In [6]:
# Ordenar por estación y fecha para aplicar forward fill correctamente
df_plata_ffill = df_plata.sort_values(['ESTACION', 'FECHA']).copy()
df_plata_ffill.update(df_plata.groupby('ESTACION').ffill())

# Vista previa de ejemplo tras forward fill
print("\nEjemplo de datos tras forward fill:")
print(df_plata_ffill.head())


Ejemplo de datos tras forward fill:
        FECHA         ESTACION  TEMP_MEAN  TEMP_MIN  TEMP_MAX  PNM_MEAN  PNM_MIN  PNM_MAX  HUM_MEAN  HUM_MIN  HUM_MAX  WIND_DIR_MEAN  WIND_DIR_MIN  WIND_DIR_MAX  WIND_SPEED_MEAN  WIND_SPEED_MIN  WIND_SPEED_MAX  TEMP_MEAN_NORM  PNM_MEAN_NORM  HUM_MEAN_NORM  WIND_DIR_MEAN_NORM  WIND_SPEED_MEAN_NORM
0  2024-06-01  CORRIENTES AERO       19.2      14.4      25.7    1014.3   1012.8   1016.4      78.3     54.0     98.0           66.5          30.0         990.0             11.8             4.0            22.0        0.481818       0.411243       0.676609            0.164437              0.275510
5  2024-06-02  CORRIENTES AERO       21.0      15.6      28.0    1013.1   1011.6   1016.7      76.6     47.0     94.0           82.1          20.0         160.0             10.7             4.0            22.0        0.536364       0.375740       0.649922            0.210824              0.247449
10 2024-06-03  CORRIENTES AERO       14.6      10.0      21.4    1023

### Imputación con la media de cada estación (solo para columnas numéricas)

In [7]:
# Imputar con la media por estación
columnas_a_imputar = ['TEMP_MEAN', 'PNM_MEAN', 'HUM_MEAN', 'WIND_SPEED_MEAN', 'WIND_DIR_MEAN']

for col in columnas_a_imputar:
    df_plata_ffill[col] = df_plata_ffill.groupby('ESTACION')[col].transform(lambda x: x.fillna(x.mean()))

# Verificar resultado tras imputación
print("\nValores nulos después de imputación con medias:")
print(df_plata_ffill[columnas_a_imputar].isnull().sum())


Valores nulos después de imputación con medias:
TEMP_MEAN          0
PNM_MEAN           0
HUM_MEAN           0
WIND_SPEED_MEAN    0
WIND_DIR_MEAN      0
dtype: int64


Estas estrategias permiten garantizar que las variables derivadas a construir se basen en datos consistentes, sin afectar la distribución ni introducir sesgos evidentes.

### Tratamiento de datos faltantes en el dataset horario

In [8]:
# Detectar horarios reales de cada estación
df_horario['HORA'] = df_horario['FECHA_HORA'].dt.hour
horarios_por_estacion = df_horario.groupby('NOMBRE')['HORA'].value_counts().unstack(fill_value=0)
horarios_mas_frecuentes = horarios_por_estacion.idxmax(axis=1)

# Detectar horarios outlier (menos del 5% de los días)
outliers_horarios = {}
for estacion in horarios_por_estacion.index:
    total_dias = df_horario[df_horario['NOMBRE'] == estacion]['FECHA_HORA'].dt.date.nunique()
    outliers = horarios_por_estacion.loc[estacion][
        horarios_por_estacion.loc[estacion] / total_dias < 0.05
    ].index.tolist()
    if outliers:
        outliers_horarios[estacion] = outliers

# Crear index completo por estación y sus horarios típicos
df_horario['FECHA'] = df_horario['FECHA_HORA'].dt.floor('D')
estaciones_h = df_horario['NOMBRE'].unique()
fecha_h_min = df_horario['FECHA'].min()
fecha_h_max = df_horario['FECHA'].max()
rango_fechas = pd.date_range(start=fecha_h_min, end=fecha_h_max, freq='D')

# Crear combinaciones válidas por estación
porcentaje_frecuencia = 0.05 # al menos en 5% de los días

index_completo_personalizado = []
for estacion in estaciones_h:
    total_dias_estacion = df_horario[df_horario['NOMBRE'] == estacion]['FECHA'].nunique()
    horas_validas = horarios_por_estacion.columns[
        (horarios_por_estacion.loc[estacion] / total_dias_estacion) >= porcentaje_frecuencia  
    ].tolist()

    for fecha in rango_fechas:
        for hora in horas_validas:
            index_completo_personalizado.append((estacion, pd.Timestamp(fecha + pd.Timedelta(hours=hora))))

index_completo_h = pd.MultiIndex.from_tuples(index_completo_personalizado, names=['NOMBRE', 'FECHA_HORA'])

# Reindexar para insertar valores faltantes en los horarios esperados únicamente
df_horario_completo = df_horario.set_index(['NOMBRE', 'FECHA_HORA']).reindex(index_completo_h).reset_index()

# Verificación
print("\nDiferencia de tamaño (horas originales vs completadas por horario habitual):")
print("Original:", len(df_horario))
print("Completo:", len(df_horario_completo))
print("\nEjemplo de datos horarios con NaN insertados:")
print(df_horario_completo[df_horario_completo.isnull().any(axis=1)].head())


Diferencia de tamaño (horas originales vs completadas por horario habitual):
Original: 43002
Completo: 43452

Ejemplo de datos horarios con NaN insertados:
        NOMBRE          FECHA_HORA FECHA  HORA  TEMP  HUM  PNM  DD  FF  estacion_archivo
39   ITUZAINGO 2024-06-03 19:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
124  ITUZAINGO 2024-06-09 20:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
423  ITUZAINGO 2024-07-01 11:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
590  ITUZAINGO 2024-07-13 10:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
716  ITUZAINGO 2024-07-22 10:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN


### Mostrar horarios outliers detectados

In [9]:
# Visualizar registros reales en horarios atípicos detectados
print("\n Registros reales en horarios atípicos:")
for estacion, horas in outliers_horarios.items():
    print(f" - {estacion}: {horas}")

# Registrar los registros reales que ocurren en horarios atípicos
df_outliers_registros = []
for estacion, horas_outlier in outliers_horarios.items():
    registros_outlier = df_horario[
        (df_horario['NOMBRE'] == estacion) &
        (df_horario['HORA'].isin(horas_outlier))
    ]
    if not registros_outlier.empty:
        df_outliers_registros.append(registros_outlier)

# Concatenar y exportar si hay registros
if df_outliers_registros:
    df_outliers_concat = pd.concat(df_outliers_registros)
    archivo_outliers = PLATA_DIR / "registros_horarios_atipicos.csv"
    df_outliers_concat.to_csv(archivo_outliers, index=False)
    print("\n Archivo exportado con registros reales en horarios atípicos:")
    print(archivo_outliers)


 Registros reales en horarios atípicos:
 - ITUZAINGO: [0, 1, 2, 3, 4, 5, 7, 8, 22, 23]
 - MERCEDES AERO (CTES): [0, 1, 2, 3, 4, 5, 22, 23]


## Exportar datasets intermedios (antes de procesar los NaN)

In [10]:
# Exportar datasets intermedios (si se desea conservar)
df_plata.to_csv(PLATA_DIR / "dataset_intermedio_horario_con_nan.csv", index=False)
df_plata_ffill.to_csv(PLATA_DIR / "dataset_intermedio_horario_ffill.csv", index=False)
df_horario_completo.to_csv(PLATA_DIR / "dataset_intermedio_horario_completo.csv", index=False)

print("Archivos generados correctamente")

Archivos generados correctamente


In [11]:
print(df_horario_completo)

                     NOMBRE          FECHA_HORA      FECHA  HORA  TEMP   HUM     PNM     DD    FF  estacion_archivo
0                 ITUZAINGO 2024-06-01 06:00:00 2024-06-01   6.0  15.8  87.0  1015.1  360.0  11.0        20240601.0
1                 ITUZAINGO 2024-06-01 09:00:00 2024-06-01   9.0  17.8  73.0  1016.0   50.0  15.0        20240601.0
2                 ITUZAINGO 2024-06-01 10:00:00 2024-06-01  10.0  18.4  71.0  1016.3   50.0  15.0        20240601.0
3                 ITUZAINGO 2024-06-01 11:00:00 2024-06-01  11.0  20.1  67.0  1016.1   50.0  15.0        20240601.0
4                 ITUZAINGO 2024-06-01 12:00:00 2024-06-01  12.0  21.4  64.0  1018.1   50.0  17.0        20240601.0
...                     ...                 ...        ...   ...   ...   ...     ...    ...   ...               ...
43447  MERCEDES AERO (CTES) 2025-07-31 17:00:00 2025-07-31  17.0  23.4  46.0  1010.5   20.0  26.0        20250731.0
43448  MERCEDES AERO (CTES) 2025-07-31 18:00:00 2025-07-31  18.0  21.6  

## Imputación de datos faltantes basada en promedio entre días anterior y posterior

In [12]:
# Variables a imputar
variables_objetivo = ['TEMP', 'HUM', 'PNM', 'DD', 'FF']

df_interp = df_horario_completo.copy()

# Asegurar FECHA y HORA correctas
df_interp['FECHA'] = df_interp['FECHA_HORA'].dt.date
df_interp['HORA'] = df_interp['FECHA_HORA'].dt.hour

# Ordenar por estación, fecha y hora
df_interp = df_interp.sort_values(by=['NOMBRE', 'FECHA', 'HORA'])

# Función de imputación por promedio entre día anterior y posterior
def imputar_valores(grupo):
    grupo = grupo.copy()  # para evitar advertencias de SettingWithCopy
    for var in variables_objetivo:
        for idx, fila in grupo.iterrows():
            if pd.isna(fila[var]):
                hora = fila['HORA']
                fecha = fila['FECHA']

                # Buscar el valor del día anterior
                val_ant = grupo[(grupo['HORA'] == hora) & (grupo['FECHA'] < fecha)][var].last_valid_index()
                val_ant = grupo.at[val_ant, var] if val_ant is not None else None

                # Buscar el valor del día posterior
                val_post = grupo[(grupo['HORA'] == hora) & (grupo['FECHA'] > fecha)][var].first_valid_index()
                val_post = grupo.at[val_post, var] if val_post is not None else None

                # Asignar promedio o valor disponible
                if val_ant is not None and val_post is not None:
                    grupo.at[idx, var] = round((val_ant + val_post) / 2, 1)
                elif val_ant is not None:
                    grupo.at[idx, var] = val_ant
                elif val_post is not None:
                    grupo.at[idx, var] = val_post
    return grupo

# Aplicar por estación SIN include_groups
df_interp = (
    df_interp.groupby('NOMBRE', group_keys=False)
    .apply(imputar_valores)
    .reset_index(drop=True)
)

# Redondear valores numéricos a 1 decimal
for var in variables_objetivo:
    df_interp[var] = df_interp[var].round(1)

# Ajustar tipos de columnas
df_interp['HORA'] = df_interp['HORA'].astype('int64')
if 'estacion_archivo' in df_interp.columns:
    df_interp['estacion_archivo'] = df_interp['estacion_archivo'].astype('int64', errors='ignore')

# Exportar
archivo_imputado = PLATA_DIR / "dataset_plata_horario_final.csv"
df_interp.to_csv(archivo_imputado, index=False)
print(f"Archivo exportado: {archivo_imputado}")


Archivo exportado: ../data/plata/dataset_plata_horario_final.csv


## Generar archivo diario a partir de la imputación de los datos faltantes en el dato_horario

In [13]:
# Generar dataset diario imputado (todas las estaciones)

# Agrupar por estación y fecha
df_diario_imputado = df_interp.groupby(['NOMBRE', 'FECHA']).agg(
    TEMP_MEAN=('TEMP', 'mean'),
    TEMP_MIN=('TEMP', 'min'),
    TEMP_MAX=('TEMP', 'max'),
    PNM_MEAN=('PNM', 'mean'),
    PNM_MIN=('PNM', 'min'),
    PNM_MAX=('PNM', 'max'),
    HUM_MEAN=('HUM', 'mean'),
    HUM_MIN=('HUM', 'min'),
    HUM_MAX=('HUM', 'max'),
    WIND_DIR_MEAN=('DD', 'mean'),
    WIND_DIR_MIN=('DD', 'min'),
    WIND_DIR_MAX=('DD', 'max'),
    WIND_SPEED_MEAN=('FF', 'mean'),
    WIND_SPEED_MIN=('FF', 'min'),
    WIND_SPEED_MAX=('FF', 'max')
).reset_index()

# Renombrar y ordenar
df_diario_imputado.rename(columns={'NOMBRE':'ESTACION'}, inplace=True)
df_diario_imputado['FECHA'] = pd.to_datetime(df_diario_imputado['FECHA'])
df_diario_imputado = df_diario_imputado.sort_values(by=['ESTACION','FECHA']).reset_index(drop=True)

# Ajustes de tipos y redondeo

# Redondear medias a 1 decimal
cols_float = ['TEMP_MEAN','PNM_MEAN','HUM_MEAN','WIND_DIR_MEAN','WIND_SPEED_MEAN']
df_diario_imputado[cols_float] = df_diario_imputado[cols_float].round(1)

# Convertir min y max a enteros
cols_int = [
    'TEMP_MIN','TEMP_MAX','PNM_MIN','PNM_MAX',
    'HUM_MIN','HUM_MAX',
    'WIND_DIR_MIN','WIND_DIR_MAX',
    'WIND_SPEED_MIN','WIND_SPEED_MAX'
]
df_diario_imputado[cols_int] = df_diario_imputado[cols_int].round().astype(int)

# Normalización Min-Max de las variables MEAN

variables_mean = ['TEMP_MEAN','PNM_MEAN','HUM_MEAN','WIND_DIR_MEAN','WIND_SPEED_MEAN']
for var in variables_mean:
    col_norm = var + '_NORM'
    min_val = df_diario_imputado[var].min()
    max_val = df_diario_imputado[var].max()
    df_diario_imputado[col_norm] = ((df_diario_imputado[var] - min_val) / (max_val - min_val)).round(5)

# Validación y exportación

print("\nColumnas finales:", df_diario_imputado.columns.tolist())
print("Filas:", len(df_diario_imputado), "| Columnas:", len(df_diario_imputado.columns))
print(df_diario_imputado.groupby('ESTACION').size())

# Guardar el dataset diario imputado completo
archivo_diario_imputado = PLATA_DIR / "dataset_plata_diario_final.csv"
df_diario_imputado.to_csv(archivo_diario_imputado, index=False)
print(f"Archivo diario imputado exportado: {archivo_diario_imputado}")


Columnas finales: ['ESTACION', 'FECHA', 'TEMP_MEAN', 'TEMP_MIN', 'TEMP_MAX', 'PNM_MEAN', 'PNM_MIN', 'PNM_MAX', 'HUM_MEAN', 'HUM_MIN', 'HUM_MAX', 'WIND_DIR_MEAN', 'WIND_DIR_MIN', 'WIND_DIR_MAX', 'WIND_SPEED_MEAN', 'WIND_SPEED_MIN', 'WIND_SPEED_MAX', 'TEMP_MEAN_NORM', 'PNM_MEAN_NORM', 'HUM_MEAN_NORM', 'WIND_DIR_MEAN_NORM', 'WIND_SPEED_MEAN_NORM']
Filas: 2130 | Columnas: 22
ESTACION
CORRIENTES AERO            426
ITUZAINGO                  426
MERCEDES AERO (CTES)       426
MONTE CASEROS AERO         426
PASO DE LOS LIBRES AERO    426
dtype: int64
Archivo diario imputado exportado: ../data/plata/dataset_plata_diario_final.csv


## Verificar las imputaciones

In [14]:
# Verificación de imputación final
print("Valores restantes faltantes por variable:")
print(df_interp[variables_objetivo].isnull().sum())

# Vista previa de algunos valores aún faltantes (si existen)
print("\nEjemplos de filas con valores aún faltantes:")
print(df_interp[df_interp[variables_objetivo].isnull().any(axis=1)].head())

Valores restantes faltantes por variable:
TEMP    0
HUM     0
PNM     0
DD      0
FF      0
dtype: int64

Ejemplos de filas con valores aún faltantes:
Empty DataFrame
Columns: [NOMBRE, FECHA_HORA, FECHA, HORA, TEMP, HUM, PNM, DD, FF, estacion_archivo]
Index: []


# Contar imputaciones por columna

In [15]:
imputaciones = {}
for var in variables_objetivo:
    # Detectar índices donde original es NaN pero imputado tiene valor
    mask_imputado = df_horario_completo[var].isna() & df_interp[var].notna()
    imputaciones[var] = mask_imputado.sum()

# Mostrar resumen
print("Resumen de imputaciones por variable:")
for var, count in imputaciones.items():
    print(f" - {var}: {count} valores imputados")

# Mostrar ejemplos comparativos (solo filas donde hubo imputación)
print("\nEjemplos de imputaciones realizadas:")
for var in variables_objetivo:
    mask = df_horario_completo[var].isna() & df_interp[var].notna()
    if mask.any():
        print(f"\nVariable: {var}")
        print(df_interp.loc[mask, ['FECHA_HORA', 'NOMBRE', var]])

Resumen de imputaciones por variable:
 - TEMP: 450 valores imputados
 - HUM: 450 valores imputados
 - PNM: 450 valores imputados
 - DD: 450 valores imputados
 - FF: 450 valores imputados

Ejemplos de imputaciones realizadas:

Variable: TEMP
               FECHA_HORA                   NOMBRE  TEMP
39    2024-06-02 15:00:00          CORRIENTES AERO  27.8
124   2024-06-06 04:00:00          CORRIENTES AERO  19.9
423   2024-06-18 15:00:00          CORRIENTES AERO  21.5
590   2024-06-25 14:00:00          CORRIENTES AERO  13.4
716   2024-06-30 20:00:00          CORRIENTES AERO  11.1
...                   ...                      ...   ...
40312 2025-03-23 04:00:00  PASO DE LOS LIBRES AERO  20.4
40313 2025-03-23 05:00:00  PASO DE LOS LIBRES AERO  19.9
40314 2025-03-23 06:00:00  PASO DE LOS LIBRES AERO  19.6
40315 2025-03-23 07:00:00  PASO DE LOS LIBRES AERO  20.0
42057 2025-06-03 21:00:00  PASO DE LOS LIBRES AERO  15.8

[450 rows x 3 columns]

Variable: HUM
               FECHA_HORA           

# Visualización de imputación de los días faltantes

In [16]:
# Carpeta con los archivos de días faltantes
FALTANTES_DIR = Path("../data/faltantes")  # Ajustar al directorio correcto
variables_objetivo = ['TEMP', 'HUM', 'PNM', 'DD', 'FF']

# Función para leer días faltantes de un archivo
def leer_dias_faltantes(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    fechas = [line.strip() for line in lines if line.strip() and line.strip()[0].isdigit()]
    return pd.to_datetime(fechas).date

# Recorrer todos los archivos .txt de días faltantes
faltantes_files = list(FALTANTES_DIR.glob("*.txt"))

resumen_resultados = []

for file_path in faltantes_files:
    estacion = file_path.stem.replace("dias_faltantes_", "").replace("_", " ").upper()
    dias_faltantes = leer_dias_faltantes(file_path)
    
    print(f"\n=== Estación: {estacion} ===")
    resultados_estacion = {"Estación": estacion, "Total días faltantes": len(dias_faltantes), "Días completos": 0, "Días con NaN": 0}
    
    for fecha in dias_faltantes:
        subset_original = df_horario_completo[(df_horario_completo['NOMBRE'].str.upper() == estacion) & (df_horario_completo['FECHA'] == fecha)]
        subset_imputado = df_interp[(df_interp['NOMBRE'].str.upper() == estacion) & (df_interp['FECHA'] == fecha)]
        
        if subset_imputado[variables_objetivo].isnull().any().any():
            resultados_estacion["Días con NaN"] += 1
            print(f"\nFecha {fecha} aún con NaN")
        else:
            resultados_estacion["Días completos"] += 1
            print(f"\nFecha {fecha} imputada correctamente")
        
        # Comparar antes y después
        if not subset_imputado.empty:
            print("\n--- Antes (Original con NaN) ---")
            print(subset_original[['FECHA_HORA', 'NOMBRE'] + variables_objetivo])
            print("\n--- Después (Imputado) ---")
            print(subset_imputado[['FECHA_HORA', 'NOMBRE'] + variables_objetivo])
    
    resumen_resultados.append(resultados_estacion)

# Mostrar resumen final
df_resumen = pd.DataFrame(resumen_resultados)
print("\nResumen de verificación de imputaciones por estación:")
print(df_resumen)


=== Estación: PASO DE LOS LIBRES AERO ===

Fecha 2024-10-24 imputada correctamente

--- Antes (Original con NaN) ---
Empty DataFrame
Columns: [FECHA_HORA, NOMBRE, TEMP, HUM, PNM, DD, FF]
Index: []

--- Después (Imputado) ---
               FECHA_HORA                   NOMBRE  TEMP   HUM     PNM     DD    FF
36708 2024-10-24 00:00:00  PASO DE LOS LIBRES AERO  22.2  80.5  1008.0  190.0  14.0
36709 2024-10-24 01:00:00  PASO DE LOS LIBRES AERO  21.8  83.5  1007.6  160.0   7.5
36710 2024-10-24 02:00:00  PASO DE LOS LIBRES AERO  21.4  83.5  1007.3  190.0  13.0
36711 2024-10-24 03:00:00  PASO DE LOS LIBRES AERO  21.0  85.5  1006.8  190.0  12.0
36712 2024-10-24 04:00:00  PASO DE LOS LIBRES AERO  20.8  87.0  1006.6  180.0  14.0
36713 2024-10-24 05:00:00  PASO DE LOS LIBRES AERO  20.5  88.5  1006.8  180.0  14.0
36714 2024-10-24 06:00:00  PASO DE LOS LIBRES AERO  20.2  89.0  1007.0  180.0  15.0
36715 2024-10-24 07:00:00  PASO DE LOS LIBRES AERO  20.9  88.5  1007.4  190.0  12.0
36716 2024-10-24 0

# Verificación de datos faltantes

In [17]:
# Crear un DataFrame reducido solo con días únicos por estación
df_fechas = df_interp[['FECHA', 'NOMBRE']].drop_duplicates()

# Generar rango completo de fechas
fechas_totales = pd.date_range(start=df_fechas['FECHA'].min(), end=df_fechas['FECHA'].max(), freq='D')

# Estaciones
estaciones = df_fechas['NOMBRE'].unique()
# Crear todas las combinaciones posibles (fecha, estación)
index_completo = pd.MultiIndex.from_product([fechas_totales, estaciones], names=['FECHA', 'NOMBRE'])

# Reindexar
df_check = df_fechas.set_index(['FECHA', 'NOMBRE']).reindex(index_completo).reset_index()

# Verificar faltantes
faltantes = df_check[df_check.isnull().any(axis=1)][['NOMBRE', 'FECHA']]

# Exportar resultados
if not faltantes.empty:
    archivo_faltantes_final = PLATA_DIR / "fechas_faltantes_post_imputacion.txt"
    faltantes.to_csv(archivo_faltantes_final, index=False, sep='\t')
    print(f"Fechas faltantes exportadas a: {archivo_faltantes_final}")
else:
    print("No se encontraron fechas faltantes después de la imputación.")

# Vista rápida
print("\nEjemplo de fechas faltantes:")
print(faltantes.head())

No se encontraron fechas faltantes después de la imputación.

Ejemplo de fechas faltantes:
Empty DataFrame
Columns: [NOMBRE, FECHA]
Index: []


# Conclusión

En este notebook hemos completado el proceso de **enriquecimiento de la Capa Plata**.

Se garantiza:
- **Datos horarios completos** por estación para el período de análisis.
- **Tratamiento adecuado de valores faltantes**, aplicando imputaciones coherentes con el comportamiento histórico de cada estación.
- Generación de un **dataset final imputado** que sirve como base para análisis avanzados (clustering, PCA, detección de eventos).

Con esta preparación, los datos están listos para las tareas de **minería de datos y categorización**, que abordaremos en la clase siguiente.